# Ourself

## Notes from Bevan
Here is a link to our google drive folder with the Ourself data. There are two files: one for the installation that is still up at Roosevelt Plaza Park, one for the installation that was up at the Kroc Center. 

The file contains data in the following format:
    
    datestamp, timestamp, behavior

The software was running a state machine that tracks what it thinks people are doing so it knows how to control the lights and sounds; the logger recorded the status of that state machine. It recorded a data point every second as long as a sensor is tripped. 
The sensors are microwave sensors to detect motion, and weight sensors to detect when someone is on the central platform. 

If a sensor trip awakens it from a null state, it records this as someone arriving - "ARR".
If the weight sensors are tripped, it records this as occupied - "OCC".
If the weight sensors disengage and then microwave sensors are activated, it interprets this as someone departing - "DEP"

There are of course ambiguities in the interpretation. Someone may have left the platform while someone else entered. Sometime it may flip from "DEParted" straight to "OCCupied" again without hitting "ARRival", such as if there were many people milling around the sculpture. 

We are interested in overall trends, especially the following:

    - on how many occasions was the platform occupied?
    - how long was the total time of occupation?
    - how long was the average time of occupation?
    - what was the ratio of "ARR" + "DEP" times to the "OCC" time?

In [1]:
import pandas as pd

In [63]:
# get data
df = pd.read_csv(
                 '../data/Ourself_datalog_KROC_03-13-2017.TXT'
                 ,names = ['dateTime', 'ARR', 'OCC', 'DEP']
                 ,dtype = {'dateTime':object, 'ARR':str, 'OCC':str, 'DEP':str}
)
df.fillna(0, inplace = True)

df['arrived'] = df.ARR == 'ARR'
df['occupied'] = df.OCC == 'OCC'
df['departed'] = df.DEP == 'DEP'
df.index = pd.to_datetime(df.dateTime)

df = df[['arrived', 'occupied', 'departed']]

In [72]:
df[(df.departed == True) & (df.occupied == True)]

,arrived,occupied,departed
dateTime,,,
2016-05-10 15:53:01,True,True,True
2016-05-10 15:53:03,True,True,True
2016-05-10 15:53:04,True,True,True
2016-05-10 15:53:05,True,True,True
2016-05-10 15:53:17,True,True,True
2016-05-10 15:53:18,True,True,True
2016-05-10 15:53:19,True,True,True
2016-05-10 15:53:21,True,True,True
2016-05-10 15:53:25,True,True,True
